In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import validation_curve
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
%pwd

'/Users/ryanmaki'

In [3]:
bank_bal=pd.read_csv('OBA_410_Group_Data.csv')
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,0,0.000000,0.006923,0.000000,0.593501,0.593551,0.998775,0.795533,0.807547,0.300687,...,0.224792,2.090270e-04,0.623797,0.593502,0.770902,0.285511,0.026790,0.565152,1,0.019171
1,1,0.024277,0.000000,0.033514,0.591324,0.591324,0.994015,0.791503,0.803775,0.303602,...,0.584583,2.710000e+09,0.623818,0.591322,0.827165,0.276846,0.026791,0.565158,1,0.054959
2,1,0.066933,0.057185,0.054821,0.601861,0.601861,0.998825,0.796779,0.808717,0.302760,...,0.525651,5.803037e-03,0.623648,0.601857,1.000000,0.182790,0.026763,0.565021,1,0.009178
3,0,0.081412,0.091256,0.091761,0.602935,0.602935,0.973875,0.767759,0.781429,0.304248,...,0.412621,1.920000e+09,0.623783,0.602932,0.813135,0.278182,0.026791,0.565158,1,0.037886
4,0,0.082826,0.069287,0.088388,0.646846,0.646846,0.997083,0.795795,0.807815,0.304684,...,0.000000,0.000000e+00,0.623730,0.646841,0.748252,0.284685,0.026791,0.565158,1,0.019908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.787988,0.804514,0.783928,0.608015,0.608015,0.999147,0.798222,0.809903,0.304606,...,0.905790,2.948022e-03,0.624791,0.608017,0.847232,0.277502,0.026791,0.565159,1,0.044549
6815,0,0.818018,0.801570,0.814551,0.634983,0.634983,0.999278,0.797758,0.809581,0.303522,...,0.923155,1.093630e-04,0.623786,0.634983,0.848965,0.278178,0.026794,0.565170,1,0.037918
6816,0,0.864964,0.942706,0.932598,0.622371,0.622371,0.999308,0.797793,0.809673,0.303519,...,0.959320,2.271900e-04,0.624001,0.622370,0.849236,0.276178,0.026791,0.565158,1,0.074893
6817,0,0.971530,1.000000,1.000000,0.640035,0.640035,0.999571,0.798105,0.809906,0.303516,...,0.981315,1.534960e-04,0.624305,0.640035,0.851547,0.277013,0.026799,0.565196,1,0.051752


# Data Sampling

In [4]:
bank_bal=bank_bal.groupby('Bankrupt?',group_keys=False).apply(lambda x: x.sample(1000,replace=True,random_state=0))
bank_bal['Bankrupt?'].value_counts()

0    1000
1    1000
Name: Bankrupt?, dtype: int64

In [5]:
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2935,0,0.495686,0.550916,0.542963,0.599209,0.599209,0.999001,0.797404,0.809320,0.303483,...,0.804195,5.797642e-03,0.623865,0.599205,0.840688,0.279388,0.027243,0.566668,1,0.030801
2807,0,0.494028,0.556149,0.539697,0.603273,0.603021,0.999016,0.797448,0.809353,0.303527,...,0.806766,1.678171e-03,0.623481,0.603273,0.841082,0.281947,0.027068,0.566204,1,0.023555
1834,0,0.478672,0.541976,0.531881,0.605226,0.605226,0.999008,0.797419,0.809341,0.303494,...,0.801978,4.771583e-03,0.623800,0.605225,0.840553,0.280316,0.026973,0.565898,1,0.027411
3475,0,0.503924,0.551297,0.549280,0.607064,0.607064,0.998987,0.797522,0.809440,0.303718,...,0.804269,4.786887e-02,0.624181,0.607060,0.840859,0.282237,0.027089,0.566266,1,0.023043
5150,0,0.536635,0.597580,0.580813,0.610466,0.610141,0.999138,0.797629,0.809496,0.303588,...,0.832874,3.267898e-03,0.625247,0.610468,0.842125,0.276202,0.026792,0.565160,1,0.073883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,1,0.454346,0.513792,0.506023,0.597522,0.597522,0.998899,0.797237,0.809064,0.303405,...,0.771996,6.000143e-03,0.621979,0.597522,0.837544,0.283088,0.026463,0.562888,1,0.021747
1791,1,0.477990,0.538323,0.529632,0.616966,0.616930,0.999190,0.797473,0.809361,0.303207,...,0.797931,2.171998e-02,0.619427,0.616962,0.840159,0.276686,0.027789,0.567605,1,0.058527
368,1,0.417540,0.470726,0.460892,0.600852,0.600852,0.998935,0.797160,0.809086,0.303195,...,0.754696,8.433273e-03,0.624491,0.600848,0.836693,0.279759,0.026550,0.563668,1,0.029296
1250,1,0.466728,0.490078,0.501097,0.607850,0.607850,0.998855,0.797196,0.809109,0.303425,...,0.767057,8.094303e-03,0.623321,0.607845,0.834982,0.293298,0.026485,0.563103,1,0.015432


# Logistic Regression¶

In [6]:
bank_bal[bank_bal['Bankrupt?']==1].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,1.0,0.42,0.46,0.46,0.60,0.60,1.00,0.80,0.81,0.30,...,0.74,5.630000e+07,0.62,0.60,0.83,0.29,0.03,0.56,1.0,0.02
std,0.0,0.08,0.11,0.09,0.01,0.01,0.00,0.00,0.00,0.01,...,0.08,5.971907e+08,0.01,0.01,0.06,0.05,0.02,0.01,0.0,0.05
min,1.0,0.02,0.00,0.03,0.53,0.53,0.99,0.76,0.76,0.24,...,0.41,0.000000e+00,0.59,0.53,0.00,0.00,0.02,0.55,1.0,0.00
25%,1.0,0.39,0.43,0.44,0.59,0.59,1.00,0.80,0.81,0.30,...,0.72,0.000000e+00,0.62,0.59,0.83,0.28,0.03,0.56,1.0,0.02
50%,1.0,0.44,0.49,0.49,0.60,0.60,1.00,0.80,0.81,0.30,...,0.77,0.000000e+00,0.62,0.60,0.84,0.29,0.03,0.56,1.0,0.02
75%,1.0,0.47,0.53,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.79,1.000000e-02,0.62,0.60,0.84,0.29,0.03,0.57,1.0,0.02
max,1.0,0.58,0.58,0.62,0.67,0.67,1.00,0.80,0.81,0.31,...,0.82,9.170000e+09,0.68,0.67,1.00,1.00,0.26,0.62,1.0,0.80


In [7]:
bank_bal[bank_bal['Bankrupt?']==0].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
std,0.0,0.06,0.06,0.06,0.01,0.01,0.01,0.02,0.03,0.01,...,0.04,0.04,0.01,0.01,0.00,0.01,0.02,0.01,0.0,0.06
min,0.0,0.00,0.01,0.00,0.45,0.45,0.61,0.16,0.00,0.00,...,0.22,0.00,0.55,0.45,0.77,0.27,0.00,0.33,1.0,0.01
25%,0.0,0.48,0.54,0.53,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.03,0.57,1.0,0.03
50%,0.0,0.50,0.56,0.55,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.00,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.04
75%,0.0,0.54,0.59,0.59,0.61,0.61,1.00,0.80,0.81,0.30,...,0.83,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.06
max,0.0,1.00,0.95,0.96,0.66,0.66,1.00,0.81,0.82,0.33,...,0.94,0.54,0.70,0.66,0.85,0.33,0.54,0.74,1.0,1.00


## controlling the complexity of log regression, using parameter 'c'

In [9]:
bank_bal=bank_bal.sample(frac=1, random_state=0)
bank_bal.head(1)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
722,0,0.44484,0.548572,0.491086,0.603987,0.603987,0.998982,0.797403,0.809329,0.303522,...,0.799526,0.000498,0.624015,0.603983,0.840376,0.281577,0.028708,0.568407,1,0.02427


In [10]:
X_bank, y_bank=bank_bal.iloc[:,1:], bank_bal['Bankrupt?'] 
display(X_bank.head(2))
display(y_bank.head(2))

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
722,0.444840,0.548572,0.491086,0.603987,0.603987,0.998982,0.797403,0.809329,0.303522,0.781454,...,0.799526,0.000498,0.624015,0.603983,0.840376,0.281577,0.028708,0.568407,1,0.024270
1193,0.464876,0.527638,0.517640,0.615885,0.615979,0.998964,0.797347,0.809289,0.303462,0.781545,...,0.793192,0.000484,0.623923,0.615882,0.839844,0.278318,0.026505,0.563292,1,0.036848


722     0
1193    0
Name: Bankrupt?, dtype: int64

## Feature Selection

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
dt=DecisionTreeClassifier(random_state=0)
dt.fit(X_bank,y_bank)

DecisionTreeClassifier(random_state=0)

In [13]:
sum(dt.feature_importances_>.01)

16

In [14]:
X_bank_sel=X_bank[X_bank.columns[dt.feature_importances_>.01]]
X_bank_sel.head()

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
722,0.781454,1.536810e-04,0.000750,0.168935,0.203933,0.848202,1.147600e-04,0.008301,0.633244,0.009373,0.001232,0.392834,0.009797,1.608880e-04,0.671612,0.568407
1193,0.781545,3.643710e-04,0.000179,0.172306,0.213009,0.847923,6.300000e+09,0.012370,0.628167,0.004880,0.000416,0.392031,0.010332,4.460000e+09,0.671543,0.563292
895,0.781277,8.270000e+09,0.000726,0.156799,0.200151,0.848116,5.960000e+09,0.001560,0.625068,0.022234,0.001487,0.390177,0.059424,1.400000e+09,0.671572,0.553328
543,0.781034,8.084150e-04,0.000541,0.156968,0.194573,0.848080,3.700000e+09,0.009433,0.630274,0.006178,0.000243,0.382251,0.000385,9.330000e+07,0.671565,0.564618
1066,0.781485,1.277780e-04,0.000144,0.177574,0.204500,0.847894,5.760000e+09,0.008376,0.630570,0.004017,0.000728,0.390810,0.004908,9.340000e+09,0.671523,0.565096


## transform the features

In [15]:
scaler1=MinMaxScaler()
X_bank_trns=scaler1.fit_transform(X_bank_sel)
X_bank_trns

array([[9.78590119e-01, 5.41422805e-15, 7.81328125e-13, ...,
        1.61049049e-14, 9.72166248e-01, 5.84139837e-01],
       [9.78727683e-01, 2.66745711e-14, 1.86477083e-13, ...,
        4.46446446e-01, 9.72067034e-01, 5.71521463e-01],
       [9.78322835e-01, 8.34510595e-01, 7.56326042e-13, ...,
        1.40140140e-01, 9.72107797e-01, 5.46943766e-01],
       ...,
       [9.78336190e-01, 8.34510595e-01, 8.12581250e-13, ...,
        2.43243243e-01, 9.72113628e-01, 5.69557340e-01],
       [9.79000691e-01, 8.68143290e-15, 0.00000000e+00, ...,
        2.25126126e-14, 9.72166854e-01, 5.76127636e-01],
       [9.75526842e-01, 1.09743794e-13, 1.13136354e-12, ...,
        2.52252252e-02, 9.72066090e-01, 5.73655402e-01]])

## validation curve functions

In [16]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_trns, y_bank, test_size=0.2, random_state=0)

In [17]:
train_scores_log_reg, test_scores_log_reg=validation_curve(LogisticRegression(penalty='l1', solver='saga', max_iter=100000), X_train, y_train, param_name='C',
                                           param_range=[0.01,0.05,1,5,10,20], cv=4)

In [18]:
train_scores_log_reg.mean(axis=1).round(4)

array([0.505 , 0.53  , 0.7977, 0.804 , 0.8069, 0.809 ])

In [19]:
test_scores_log_reg.mean(axis=1).round(4)

array([0.505 , 0.53  , 0.7912, 0.7944, 0.7981, 0.7988])

In [20]:
log_reg=LogisticRegression(max_iter=1000, C=100)
log_reg.fit(X_train, y_train)

LogisticRegression(C=100, max_iter=1000)

In [21]:
print('log reg acc on train: {:.2%}'.format(log_reg.score(X_train, y_train)))
print('log reg acc on test: {:.2%}'.format(log_reg.score(X_test, y_test)))

log reg acc on train: 80.12%
log reg acc on test: 78.25%


In [22]:
log_reg.coef_

array([[  0.08621371,   0.67674489,  -2.04376715, -15.01933999,
        -57.39013621,  -1.71995573,   0.18203807,  -0.18155298,
          4.49743814,   3.99010718,  -3.64183006,   2.67650135,
          0.97528842,  -0.34914391,  -1.97086834,  -1.18390196]])

In [23]:
p1=[0.2,0.4,0.5,0.1,0.4,0.3,0.2,0.5,0.2,0.5,0.5,0.2,0.2,0.4,0.5,0.2,0.1]
log_reg.predict([p1])

ValueError: X has 17 features, but LogisticRegression is expecting 16 features as input.

In [24]:
import numpy as np
np.set_printoptions(suppress=True)

In [25]:
log_reg.predict_proba([p1]).round(4)

ValueError: X has 17 features, but LogisticRegression is expecting 16 features as input.

In [26]:
log_reg.classes_

array([0, 1])

In [27]:
log_reg.coef_.round(4)

array([[  0.0862,   0.6767,  -2.0438, -15.0193, -57.3901,  -1.72  ,
          0.182 ,  -0.1816,   4.4974,   3.9901,  -3.6418,   2.6765,
          0.9753,  -0.3491,  -1.9709,  -1.1839]])

In [28]:
log_reg.intercept_.round(4)

array([16.5322])

# SVM 

In [29]:
X_bank_trns

array([[0.97859012, 0.        , 0.        , ..., 0.        , 0.97216625,
        0.58413984],
       [0.97872768, 0.        , 0.        , ..., 0.44644645, 0.97206703,
        0.57152146],
       [0.97832283, 0.8345106 , 0.        , ..., 0.14014014, 0.9721078 ,
        0.54694377],
       ...,
       [0.97833619, 0.8345106 , 0.        , ..., 0.24324324, 0.97211363,
        0.56955734],
       [0.97900069, 0.        , 0.        , ..., 0.        , 0.97216685,
        0.57612764],
       [0.97552684, 0.        , 0.        , ..., 0.02522523, 0.97206609,
        0.5736554 ]])

In [30]:
svm1=LinearSVC(random_state=0)
svm1.fit(X_train,y_train)

LinearSVC(random_state=0)

In [31]:
svm1.coef_

array([[  1.16142882,   0.22542692,  -0.70742259,  -6.6900538 ,
        -10.34391391,  -0.09611625,  -0.01874457,  -0.03648844,
          0.93411752,   0.61245315,  -0.66510215,  -1.08553445,
          0.21106825,  -0.11942111,   1.35296643,  -0.3107556 ]])

In [32]:
svm1.intercept_

array([1.48482258])

In [33]:
print('svm acc on train: {:.2%}'.format(svm1.score(X_train, y_train)))
print('svm acc on test: {:.2%}'.format(svm1.score(X_test, y_test)))

svm acc on train: 78.25%
svm acc on test: 78.50%


In [34]:
b=[[0.616,0.999,0.781,0.00001,0.4719,0.0005,0.6306,0.3696,0.0415,0.0022,0.2770,0.9532,0,1.1,0.7021,0.0029,0.0345]]

In [35]:
b_trns=scaler1.transform(b)
b_trns

/Users/ryanmaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: X has 17 features, but MinMaxScaler is expecting 16 features as input.

In [36]:
svm1.predict(b_trns)

NameError: name 'b_trns' is not defined

In [37]:
svm1.intercept_.round(2)

array([1.48])

In [38]:
svm1.coef_.round(2)

array([[  1.16,   0.23,  -0.71,  -6.69, -10.34,  -0.1 ,  -0.02,  -0.04,
          0.93,   0.61,  -0.67,  -1.09,   0.21,  -0.12,   1.35,  -0.31]])

## controlling the complexity of svm by change parameter 'C'

In [39]:
LinearSVC()

LinearSVC()

In [40]:
C_range=[.1,1,5,10,50,100]

In [41]:
train_scores_SVM, test_scores_SVM=validation_curve(LinearSVC(random_state=0, max_iter=100000), X_bank_trns, y_bank, param_name='C',
                                           param_range=[.1,1,5,10,50,100], cv=4)

In [42]:
train_scores_SVM.round(4)

array([[0.7627, 0.7533, 0.7653, 0.7607],
       [0.798 , 0.786 , 0.7767, 0.784 ],
       [0.8033, 0.8027, 0.794 , 0.8027],
       [0.8073, 0.8033, 0.792 , 0.804 ],
       [0.8113, 0.8013, 0.798 , 0.8033],
       [0.8107, 0.8013, 0.7993, 0.8033]])

In [43]:
print('ave cross in 4 folds for different C values:')
print('training:', train_scores_SVM.mean(axis=1).round(4))
print('test', test_scores_SVM.mean(axis=1).round(4))

ave cross in 4 folds for different C values:
training: [0.7605 0.7862 0.8007 0.8017 0.8035 0.8037]
test [0.7595 0.7865 0.797  0.798  0.797  0.7975]


In [44]:
train_scores_SVM.mean()

0.7926944444444445

# Decision tree

In [45]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel, y_bank, test_size=0.2, random_state=0)

In [46]:
X_train

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
1482,0.781566,4.380000e+09,0.000416,0.183094,0.214617,0.847856,7.380000e+09,0.005805,0.633084,0.005846,0.001388,0.389871,0.024223,9.540000e+09,0.671594,0.568055
360,0.780927,4.409220e-04,0.000658,0.155282,0.199206,0.848138,1.090870e-04,0.001895,0.629526,0.011730,0.000797,0.391330,0.009421,1.603750e-04,0.671559,0.563721
2621,0.781534,1.544900e-04,0.000766,0.188572,0.210740,0.848381,7.200000e+09,0.002456,0.641954,0.011887,0.001286,0.394089,0.014345,1.570000e+09,0.671560,0.571268
5823,0.781784,5.830000e+09,0.000000,0.200834,0.245816,0.848080,9.110000e+09,0.014008,0.630613,0.002307,0.000623,0.432984,0.002200,3.990000e+09,0.671580,0.565159
497,0.780954,2.331230e-04,0.000360,0.162951,0.146733,0.844743,5.860000e+09,0.004686,0.630095,0.034199,0.000601,0.371713,0.040758,2.950000e+09,0.671555,0.564454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,0.781588,1.076580e-04,0.000764,0.176394,0.216602,0.847972,6.690000e+09,0.005795,0.632922,0.005270,0.001341,0.392528,0.016312,3.340000e+09,0.671576,0.567783
5752,0.781671,1.065010e-04,0.000243,0.193460,0.241089,0.848113,9.340000e+09,0.006293,0.630937,0.010348,0.001161,0.399799,0.014587,5.910000e+09,0.671585,0.565413
1331,0.781519,1.343700e-04,0.000622,0.162825,0.211118,0.848496,6.350000e+09,0.004009,0.654626,0.010741,0.000505,0.395728,0.016692,3.840000e+09,0.671584,0.640402
212,0.781424,1.216500e-04,0.000190,0.177363,0.163657,0.848005,9.500000e+09,0.004808,0.630437,0.023196,0.000935,0.395909,0.007771,8.910000e+09,0.671582,0.564948


In [47]:
train_scores_decision_tree, test_scores_decision_tree=validation_curve(DecisionTreeClassifier(random_state=0), X_train, y_train, param_name='max_leaf_nodes', 
                                          param_range=[2,6,8,10,15,25,50], cv=5)

In [48]:
dt0=DecisionTreeClassifier(random_state=0, max_leaf_nodes=10)
dt0.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

In [49]:
print('dt0 acc on train: {:.2%}'.format(dt0.score(X_train, y_train)))
print('dt0 acc on test: {:.2%}'.format(dt0.score(X_test, y_test)))

dt0 acc on train: 91.38%
dt0 acc on test: 88.25%


In [50]:
p1=[0.616,0.999,0.781,0.0001,0.471,0.0005,0.631,0.3696,0.0415,0.0022,0.27698,0,0.953,0,0.7022,0.0029,0.0345]
dt0.predict([p1])

/Users/ryanmaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 17 features, but DecisionTreeClassifier is expecting 16 features as input.

## visulalzing our decision tree

In [ ]:
export_graphviz(dt0, out_file='dt0_vis.dot', feature_names=X_bank_sel.columns, class_names=dt0.classes_.astype(str), impurity=False, filled=True)

In [ ]:
dt0.classes_

## Feature Importance 

In [ ]:
dt0.feature_importances_.round(3)

In [ ]:
X_bank_sel.columns

In [ ]:
feat_imp=pd.DataFrame(data=dt0.feature_importances_.round(3),index=X_bank_sel.columns, columns=['importance'])
feat_imp.sort_values('importance', ascending=False)

# Random Forest

In [51]:
bank_rf=RandomForestClassifier(n_estimators=300, random_state=0)
bank_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=0)

In [52]:
print('bank_rf on train: {:.2%}'.format(bank_rf.score(X_train, y_train)))
print('bank_rf on test: {:.2%}'.format(bank_rf.score(X_test, y_test)))

bank_rf on train: 100.00%
bank_rf on test: 96.00%


## Feature Importance

In [53]:
bank_rf.feature_importances_

array([0.13153111, 0.01698596, 0.03420076, 0.0553748 , 0.13095282,
       0.02217896, 0.02702303, 0.09100713, 0.08187213, 0.17592695,
       0.02918173, 0.03776205, 0.03085517, 0.03530998, 0.03055557,
       0.06928185])

In [54]:
bank_rf_feat_imp=pd.DataFrame(data=bank_rf.feature_importances_, index=X_bank_sel.columns, columns=['importance'])
bank_rf_feat_imp.sort_values(by=['importance'], ascending=False).round(3)

,importance
Total debt/Total net worth,0.176
Continuous interest rate (after tax),0.132
Persistent EPS in the Last Four Seasons,0.131
Quick Ratio,0.091
Interest Expense Ratio,0.082
Interest Coverage Ratio (Interest expense to EBIT),0.069
Net Value Per Share (A),0.055
Operating profit per person,0.038
Cash Turnover Rate,0.035
Interest-bearing debt interest rate,0.034


In [55]:
X_test.iloc[20:25,:]

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
26,0.779592,7.320000e+09,0.000296,0.152290,0.154392,0.847927,3.140000e+09,0.001296,0.630586,0.006436,0.002037,0.392540,0.013745,3.870000e+09,0.671531,0.565131
3296,0.781585,1.270850e-04,0.000224,0.193713,0.218682,0.848250,6.460000e+09,0.004599,0.632953,0.015481,0.000605,0.395649,0.015834,1.470000e+09,0.671569,0.567642
650,0.781213,1.472050e-04,0.000897,0.143567,0.195424,0.848074,5.130000e+09,0.000942,0.629331,0.018052,0.001514,0.382454,0.030209,6.450000e+08,0.671572,0.562983
425,0.781247,1.378390e-04,0.000259,0.150352,0.185308,0.847930,5.880000e+09,0.003293,0.630231,0.028478,0.000515,0.387406,0.010175,9.240000e+09,0.671527,0.564742
9,0.781065,3.006550e-04,0.000280,0.142598,0.122341,0.846452,2.620000e+09,0.005659,0.630583,0.017482,0.000856,0.355205,0.003465,2.080000e+09,0.671547,0.565116


In [56]:
bank_rf.predict(X_test.iloc[20:25,:])

array([1, 0, 1, 1, 1])

# KNN Classifier

In [57]:
train_scores_KNN, test_scores_KNN=validation_curve(KNeighborsClassifier(), X_bank_trns, y_bank, param_name='n_neighbors', 
                                           param_range=[1,2,3,4,5,6], cv=4)

In [58]:
train_scores_KNN.mean(axis=1)

array([1.        , 0.99183333, 0.9415    , 0.93066667, 0.89666667,
       0.88133333])

In [59]:
test_scores_KNN.mean(axis=1)

array([0.932 , 0.9105, 0.882 , 0.866 , 0.8385, 0.811 ])

In [60]:
train_scores_KNN.mean()

0.9403333333333332

In [61]:
test_scores_KNN.mean()

0.8733333333333334

In [62]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel,y_bank,
                                                  random_state=0)

In [63]:
knn_cls_bank_bal=KNeighborsClassifier(n_neighbors=1)

In [64]:
knn_cls_bank_bal.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=1)

In [65]:
print('knn_cls_bank_bal acc on train: {:.2%}'.format(knn_cls_bank_bal.score(X_train,y_train)))
print('knn_cls_bank_bal acc on test: {:.2%}'.format(knn_cls_bank_bal.score(X_test,y_test)),'\n')

knn_cls_bank_bal acc on train: 99.67%
knn_cls_bank_bal acc on test: 90.80% 



In [66]:
knn_cls_bank_bal.score(X_bank_sel,y_bank)

0.9745

In [67]:
w1=[1,1,0.5,0.4,0.3,0.3,0.6,0.7,1,0.4,0.2,0.5,0.7,0.4,0.1,0.9]
knn_cls_bank_bal.predict([w1])

/Users/ryanmaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([0])

In [68]:
knn_cls_bank_bal.predict(X_bank_sel)

array([0, 1, 1, ..., 1, 0, 1])

In [69]:
len(knn_cls_bank_bal.predict(X_bank_sel))

2000

In [70]:
predict_array = knn_cls_bank_bal.predict(X_bank_sel)
count_non_zero = 0
for i in predict_array:
    if i != 1:
        count_non_zero += 1
print(count_non_zero)      

967


In [71]:
sum(knn_cls_bank_bal.predict(X_bank_sel))

1033

In [72]:
X_bank_sel.describe()

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
count,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2000.000000,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2000.000000
mean,0.780825,1.914712e+09,1.003600e+07,0.175664,0.209894,0.847405,5.193806e+09,2.978000e+07,0.630928,5.205000e+06,1.096000e+07,0.392937,7.220000e+06,2.197850e+09,0.671238,0.565183
std,0.014703,3.267464e+09,8.306775e+07,0.029689,0.039638,0.007444,2.635442e+09,5.092429e+08,0.008025,1.343253e+08,2.078417e+08,0.032600,1.687086e+08,2.583247e+09,0.015024,0.008484
min,0.133479,1.000260e-04,0.000000e+00,0.069656,0.000000,0.736430,1.002510e-04,0.000000e+00,0.525126,0.000000e+00,0.000000e+00,0.109281,0.000000e+00,0.000000e+00,0.000000,0.331613
25%,0.781286,1.584800e-04,2.537750e-04,0.155998,0.194100,0.847930,4.375000e+09,2.440152e-03,0.630050,4.754740e-03,6.391200e-04,0.385586,4.782282e-03,3.907258e-04,0.671561,0.564427
50%,0.781537,2.808230e-04,3.895390e-04,0.174835,0.211118,0.848024,5.900000e+09,4.795461e-03,0.630612,9.874247e-03,9.536080e-04,0.392834,1.003300e-02,1.150000e+09,0.671571,0.565158
75%,0.781645,3.400000e+09,6.360640e-04,0.187992,0.226151,0.848090,6.900000e+09,9.104483e-03,0.631009,1.682033e-02,1.428383e-03,0.398264,2.283310e-02,3.710000e+09,0.671579,0.565586
max,0.795631,9.910000e+09,9.600000e+08,0.534280,1.000000,0.892423,9.980000e+09,9.230000e+09,0.812165,3.470000e+09,6.490000e+09,0.978178,5.290000e+09,9.990000e+09,0.690841,0.736985
